In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

2024-03-02 23:02:10.618472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.12.0'

In [3]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')
X2 = np.load('bits-740.npy')
X3 = np.load('blosum-740.npy')


X1 = X1[:,0:T,:]
X2 = X2[:,0:T,:]
X3 = X3[:,0:T,:]


Y  = [1 for _ in range(376)]
Y += [0 for _ in range(364)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(X2.shape)
print(X3.shape)
print(Y.shape)

(740, 15, 20)
(740, 15, 31)
(740, 15, 20)
(740, 2)


In [4]:
print(X1.shape)
print(X1[0].shape)
X1[0]

(740, 15, 20)
(15, 20)


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1,

In [5]:
print(X2.shape)
print(X2[0].shape)
X2[0]

(740, 15, 31)
(15, 31)


array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0,

In [6]:
print(X3.shape)
X3[0]

(740, 15, 20)


array([[ 0, -3, -1, -2, -3,  6, -2, -4, -2, -4, -3,  0, -2, -2, -2,  0,
        -2, -3, -2, -3],
       [-1, -1, -4, -3,  0, -4, -3,  2, -2,  4,  2, -3, -3, -2, -2, -2,
        -1,  1, -2, -1],
       [-3, -2, -4, -3,  1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3,
        -2, -3, 11,  2],
       [ 1, -1,  0,  0, -2,  0, -1, -2,  0, -2, -1,  1, -1,  0, -1,  4,
         1, -2, -3, -2],
       [-1, -3, -1,  1, -3, -2, -1, -3,  5, -2, -1,  0, -1,  1,  2,  0,
        -1, -2, -3, -2],
       [-1, -1, -3, -3,  0, -4, -3,  4, -3,  2,  1, -3, -3, -3, -3, -2,
        -1,  3, -3, -1],
       [-1, -3, -1,  1, -3, -2, -1, -3,  5, -2, -1,  0, -1,  1,  2,  0,
        -1, -2, -3, -2],
       [-1, -4,  2,  5, -3, -2,  0, -3,  1, -3, -2,  0, -1,  2,  0,  0,
        -1, -2, -3, -2],
       [ 0, -1, -3, -2, -1, -3, -3,  3, -2,  1,  1, -3, -2, -2, -3, -2,
         0,  4, -3, -1],
       [ 0, -3, -1, -2, -3,  6, -2, -4, -2, -4, -3,  0, -2, -2, -2,  0,
        -2, -3, -2, -3],
       [-1, -3, -1,  1, -3, -2

In [7]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.12.0.


In [8]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)


    ### Head-2:
    # input2 = Input(shape=X2[0].shape)

    # x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input2)
    # x = BatchNormalization()(x)
    # x = Dropout(rate=0.70)(x)

    # x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    # x = BatchNormalization()(x)
    # x = Dropout(rate=0.70)(x)

    # head2 = Flatten()(x)


    ### Head-3:
    input3 = Input(shape=X3[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu',)(input3)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head3 = Flatten()(x)


    # merge
    merge = Concatenate()([head1, head3])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1, input3], outputs=[output])

In [89]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix
# blosum62 = MatrixInfo.blosum62

# def seq_to_blosum(sequence):
#     sequence = tf.strings.upper(sequence)  # Ensure sequence is uppercase
#     sequence_length = tf.shape(sequence)[0]
#     blosum_matrix = []
    
#     # Iterate over sequence elements
#     for i in range(sequence_length):
#         row = []
#         aa1 = sequence[i]
#         # Iterate over sequence elements again to compare with aa1
#         for j in range(sequence_length):
#             aa2 = sequence[j]
#             # Check if (aa1, aa2) or (aa2, aa1) is in the Blosum matrix
#             if (aa1, aa2) in blosum62:
#                 row.append(blosum62[(aa1, aa2)])
#             else:
#                 row.append(blosum62[(aa2, aa1)])
#         blosum_matrix.append(row)
    
#     print(blosum_matrix)
#     return tf.convert_to_tensor(np.array(blosum_matrix))


In [90]:
def build_generator(latent_dim, output_shape):
    model = Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
        layers.Reshape(output_shape),
        layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
        layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32))
    ])
    return model
    # inputs = layers.Input(shape=(latent_dim,))
    # x = layers.Dense(128, activation='relu')(inputs)
    # x = layers.Dense(256, activation='relu')(x)
    # x = layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid')(x)
    # x = layers.Reshape(output_shape)(x)
    # x = layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1]))(x)
    # x = layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32))(x)
    
    # # Define the custom layer
    # class CustomLayer(layers.Layer):
    #     def call(self, inputs):
    #         sequence = tf.keras.backend.eval(tf.argmax(inputs, axis=-1))  # Extract sequence from one-hot encoding
    #         blosum_matrix = seq_to_blosum(sequence)  # Generate Blosum matrix
    #         print(blosum_matrix)
    #         return tf.convert_to_tensor(np.array(blosum_matrix))  # Convert Blosum matrix to tensor
    
    # custom_output = CustomLayer()(x)
    
    # # Combine the generator output and custom output
    # combined_outputs = layers.Concatenate()([x, custom_output])
    
    # # Define the model
    # model = Model(inputs=inputs, outputs=[x, custom_output])
    
    # return model

In [91]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
output_shape = (25, 20)  # Desired output shape of the matrix

In [92]:
generator = build_generator(latent_dim, output_shape)

noise = tf.random.normal([1, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(1, 25, 20), dtype=int32, numpy=
array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [88]:
fake_bpf = generated_matrix[0].numpy()

print("Binary Profile Feature Matrix:")
print(fake_bpf.tolist())
print()

# Convert binary profile feature to amino acid sequence
sequence = bpf_to_sequence(generated_matrix[0].numpy())
print("Amino Acid Sequence:")
print(sequence)
print()

# blosum62 = substitution_matrices.load("BLOSUM62")

print("BLOSUM62 Matrix:")
fake_blosum = seq_to_blosum(sequence)
print(fake_blosum)

# Calculate physiochemical properties
# protein_seq = Seq(sequence)
# protein_analyzer = ProtParam.ProteinAnalysis(str(protein_seq))
# physiochemical_matrix = protein_analyzer.protein_scale(window=5)

# Display physiochemical properties matrix
print("Physiochemical Properties Matrix:")
# print(physiochemical_matrix)

def convert_input(generated_seq):
    fake_bpf = np.array(generated_seq)
    seq = bpf_to_sequence(fake_bpf)
    return np.array(fake_bpf), np.array(seq_to_blosum(seq))

Binary Profile Feature Matrix:
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'upper'

In [85]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [86]:
prediction = discriminator.predict([X1[:5,:,:], X3[:5,:,:]])
prediction

1/1 [==============================] - 0s 242ms/step


array([[0.10887526, 0.8911247 ],
       [0.21293242, 0.7870676 ],
       [0.1744912 , 0.8255088 ],
       [0.17307754, 0.82692254],
       [0.13898307, 0.8610169 ]], dtype=float32)

In [74]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

inputs_bpf = []
inputs_blosum = []
for seq in fake_seqs:
    bpf, blosum = convert_input(seq)
    inputs_bpf.append(bpf[:15])
    inputs_blosum.append(blosum[:15,:20])

inputs_bpf = np.array(inputs_bpf)
inputs_blosum = np.array(inputs_blosum)

# print(X3[:5,:,:].shape)
# print(X3[0])
# print('---------------')
# print(inputs_blosum.shape)
# print(inputs_blosum[0])

prediction = discriminator.predict([inputs_bpf, inputs_blosum])
prediction

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'upper'

In [75]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [76]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [77]:
def generator_loss(disc_output):
    # print(disc_output.shape)
    desired_output = np.zeros_like(disc_output)
    desired_output[:, 1] = 1
    return cross_entropy(desired_output, disc_output)

In [78]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [79]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [80]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE = 32

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [81]:
# tf.compat.v1.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()

def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence_indices = tf.argmax(binary_profile_feature, axis=-1)  # Get the index of the maximum value in each row
    sequence = tf.gather(tf.constant(list(amino_acids)), sequence_indices)  # Map indices to amino acids
    sequence = tf.strings.reduce_join(sequence, axis=-1)  # Join amino acids to form the sequence
    return sequence

@tf.function
def iterate_through_tensor(tensor):
    def body(x):
        # Do something with each element of the tensor
        print(x)
        return x  *2
    
    result = tf.map_fn(body, tensor)
    return result

# Example usage
@tf.function
def test_tensor():
    input_tensor = generator(noise, training=False)
    print(input_tensor)
    seq = bpf_to_sequence(input_tensor)
    print(seq)
    output_tensor = iterate_through_tensor(input_tensor)
    print(output_tensor)

test_tensor()

Tensor("sequential_1/lambda_21/Cast:0", shape=(5, 25, 20), dtype=int32)
Tensor("ReduceJoin/ReduceJoin:0", shape=(5,), dtype=string)
Tensor("map/while/TensorArrayV2Read/TensorListGetItem:0", shape=(25, 20), dtype=int32)
Tensor("PartitionedCall:0", shape=(5, 25, 20), dtype=int32)


In [177]:
blosum62 = MatrixInfo.blosum62


def bpf_to_seq2(bpf):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence_indices = tf.argmax(bpf, axis=-1)  # Get the index of the maximum value in each row
    sequence = tf.gather(tf.constant(list(amino_acids)), sequence_indices)  # Map indices to amino acids
    print(sequence)
    for i in range(len(sequence)):
        row = []
        for j in range(len(sequence)):
            a  = sequence[i]
            b = sequence[j]
            print(a)
            print(b)
            if (a, b) in blosum62:
                row.append(blosum62[(a, b)])zip

    return sequence

@tf.function
def seq_to_blosum2(sequence):
    blosum62 = MatrixInfo.blosum62
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

In [178]:
@tf.function
def train_step():
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape:
        fake_seqs_sym = generator(noise, training=True)
        
        inputs_bpf = []
        for i in range(0, len(fake_seqs_sym)):
            bpf = fake_seqs_sym[i]
            print('kir')
            seq = bpf_to_seq2(bpf)
            print('kir')
            inputs_bpf.append(bpf[:15])
        
        print(inputs_bpf)

        inputs_blosum = []
        # for seq in fake_seqs_sym:
        #     bpf, blosum = convert_input(seq)
        #     print(bpf)
        #     print(blosum)
        #     inputs_bpf.append(bpf[:15])
        #     inputs_blosum.append(blosum[:15,:20])


        fake_output = discriminator.predict([inputs_bpf, inputs_blosum])
        gen_loss = generator_loss(fake_output)
        print('loss', gen_loss)
            
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    for gradient in gradients_of_generator:
        print("Gradient:", gradient)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

def train(dataset = None, epochs = 50):
    for epoch in range(epochs):
        start = time.time()

        for batch in range(100):
            # train_step(batch[0], batch[1])
            train_step()

        # Generate and save some examples of generated sequences
        if (epoch + 1) % 5 == 0:
            noise = tf.random.normal([num_examples_to_generate, noise_dim])
            generated_sequences = generator(noise, training=False)
            print("Examples of generated sequences at epoch", epoch + 1)
            for i in range(num_examples_to_generate):
                print(bpf_to_sequence(generated_sequences[i].numpy()))

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [179]:
# Combine datasets X1 and X3
# combined_dataset = tf.data.Dataset.from_tensor_slices((X1, X3)).shuffle(len(X1)).batch(BATCH_SIZE)

# Train the model for 50 epochs
train(epochs=50)

kir
Tensor("GatherV2:0", shape=(25,), dtype=string)
Tensor("strided_slice_1:0", shape=(), dtype=string)
Tensor("strided_slice_2:0", shape=(), dtype=string)


TypeError: in user code:

    File "/tmp/ipykernel_17154/1695240435.py", line 12, in train_step  *
        seq = bpf_to_seq2(bpf)
    File "/tmp/ipykernel_17154/2143957139.py", line 16, in bpf_to_seq2  *
        if (a, b) in blosum62:

    TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.
